In [1]:
# import libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA

In [5]:
user_profile = pd.read_csv("data/User_Profile_Null_Handled.csv")
user_packages = pd.read_csv("data/BB_Usage_CF.csv")
product_profile = pd.read_csv("data/Product_Profile_Finalised_CSV.csv")
account_no_map = pd.read_csv("data/account_no_map_all.csv")
user_bb_map = pd.read_csv("data/User_BB_Package_Map.csv")

In [6]:
try:
    user_profile.drop(["Unnamed: 0"],axis=1,inplace=True)
    user_packages.drop(["Unnamed: 0"],axis=1,inplace=True)
    account_no_map.drop(["Unnamed: 0"],axis=1,inplace=True)
    user_bb_map.drop(["Unnamed: 0"],axis=1,inplace=True)
    product_profile.drop(["Unnamed: 0"],axis=1,inplace=True)
except:
    pass

In [ ]:
user_profile = user_profile[(user_profile["BB_Package"] != "NO_INFO")]
user_profile.reset_index(drop=True, inplace=True)

In [8]:
user_bb_map

,ACCOUNT_NUM.hash,BB_Package
0,060762f66bad1ea42b90cbd8317a0517,FTTH_WEB FAMILY PLUS
1,ec436d8c0d0c61d4fd22bc0bee8d457c,LTE_WEB STARTER
2,4794e9517260d9d48523c36ac515b501,WEB LITE
3,47bdab547848cf7025f64aefe8001882,WEB FAMILY PLUS
4,8079eca70de81c34a279fbf226cb85ec,LTE_WEB STARTER
...,...,...
32940,4d7d8ffc0de8e5db3fa2e0f78c77a543,WEB STARTER
32941,a3df6cf6840e7cb360d8e835197489ee,WEB LITE
32942,1688f8d201dec3c99a90271c4603148f,WEB PAL
32943,1688f8d201dec3c99a90271c4603148f,WEB FAMILY PLUS


In [9]:
rat_df = user_packages.merge(account_no_map, how="left", on="event_source.hash")
rat_df = rat_df.merge(user_bb_map, how="left",on="ACCOUNT_NUM.hash") # THIS SHOULD BE BASED ON EVENT SOURCE
rat_df.rename(columns={"BB_Package":"package","rating":"ratings"}, inplace=True)
rat_df = rat_df.merge(user_profile, how="left", on="ACCOUNT_NUM.hash")
rat_df.dropna(subset=["ACCOUNT_NUM.hash","package"], inplace=True)

In [11]:
rat_df.iloc[:,:90].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 214069 entries, 19 to 576666
Data columns (total 90 columns):
 #   Column                                    Non-Null Count   Dtype  
---  ------                                    --------------   -----  
 0   event_source.hash                         214069 non-null  object 
 1   year-month                                214069 non-null  int64  
 2   ratings                                   214069 non-null  float64
 3   ACCOUNT_NUM.hash                          214069 non-null  object 
 4   CR.hash                                   214069 non-null  object 
 5   package                                   214069 non-null  object 
 6   Voice_INCOMING_Categorical                174093 non-null  object 
 7   Voice_INCOMING_Scaled                     174093 non-null  float64
 8   Voice_INCOMING_Outlier                    174093 non-null  object 
 9   Location_Identifier                       174093 non-null  object 
 10  Location_Node      

### Exporting Package Names

PCA Ratings calculation

In [12]:
data_dim=rat_df.iloc[:,[29,47,52,57,72,76]]
data_dim.fillna(0, inplace=True)

### Run PCA on the data and reduce the dimensions in pca_num_components dimensions
pca = PCA(n_components=1)
pca.fit(data_dim)
reduced_data = pca.fit_transform(data_dim)
results_df = pd.DataFrame(reduced_data,columns=['pca_ratings'])

scaler = MinMaxScaler()
results_df['pca_ratings'] = scaler.fit_transform(results_df['pca_ratings'].values.reshape(-1,1))

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/pandas/core/frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [13]:
rating_df=rat_df.iloc[:,[3,5,2]]
rating_df=pd.concat([rating_df,results_df],axis=1)
rating_df.rename(columns={'usage':'ratings'},inplace=True)
rating_df.drop (subset=["ACCOUNT_NUM.hash","package","ratings"], inplace = True)
rating_df.drop_duplicates(subset=["ACCOUNT_NUM.hash","package","ratings"], inplace = True)

In [14]:
rating_df.head()

,ACCOUNT_NUM.hash,package,ratings,pca_ratings
0,NaN,NaN,NaN,0.691645
19,4c79064941253fbbcc922e263bb47077,FTTH_WEB FAMILY PLUS,0.7500,0.800616
21,4c79064941253fbbcc922e263bb47077,FTTH_WEB FAMILY PLUS,0.5625,0.000000
38,4c79064941253fbbcc922e263bb47077,FTTH_WEB FAMILY PLUS,1.0000,0.264705
51,9ca8c42ad4b1b525a75834dd40a05c10,FTTH_WEB FAMILY PLUS,0.7500,0.264705


In [16]:
# storing rating DF
rating_df.to_csv("data/azure/BB_user_ratings.csv")

## User content

In [15]:
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [16]:
user_profile = user_profile[(user_profile["BB_Package"] != "NO_INFO")]
user_profile.reset_index(drop=True, inplace=True)

In [17]:
user_profile.drop(["BB_Categorical","BB_Scaled","BIRTH_YEAR","Megaline_Month_Subscribed","LTE_Month_Subscribed","VAS_Month_Subscribed"], axis=1, inplace= True)

In [18]:
user_profile.shape

(14836, 114)

## Feature Selection

In [19]:
# data = user_profile.iloc[:,1:]

In [20]:
# mp = []

# for col in data.dtypes:
#     if col == "float64":
#         mp.append(False)
#     else:
#         mp.append(True)
        
# dm = gower.gower_matrix(data, cat_features = mp)

In [ ]:
# pc=PCA(n_components=len(mp)) 
# pc.fit(dm)

In [ ]:
#How mucb variance, captured together
# cumsum_arr = pc.explained_variance_ratio_.cumsum() 

In [ ]:
# pca_df = pd.DataFrame(data.columns.tolist(),columns=["Feature"])
# pca_df["Importance"] = cumsum_arr

In [21]:
# Storing Feature resuced User Profile content
user_profile.to_csv("data/azure/BB_user_content.csv")

## Product (package) content

In [ ]:
product_profile = pd.read_csv("data/product_catalog/Product_Profile_Finalised_CSV.csv")

In [ ]:
product_df=product_profile.iloc[:,[0,2,3,4,6,7,8,9,10,11,12,13,15,36,38,39,40,43,44]]

In [23]:
temp_package_df=rating_df["package"]

In [24]:
bb_products_df = product_df.merge(temp_package_df.drop_duplicates(), left_on=['Product_DB_ID'], right_on=['package'] ,
                   how='left', indicator=True)

In [25]:
bbp_df = bb_products_df[(bb_products_df["_merge"]=="both")]

In [26]:
bbp_df.drop(["package","_merge"],axis=1,inplace=True)
bbp_df.rename(columns={'Product_ID':'package'},inplace=True)
bbp_df.reset_index(drop=True, inplace=True)

/Users/chiranhewawitharana/mambaforge/envs/mactf/lib/python3.8/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
/Users/chiranhewawitharana/mambaforge/envs/mactf/lib/python3.8/site-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [27]:
cols =["BB_Data_Standard","BB_Data_Free","BB_Data_Anytime","BB_Data_Unlimited"]

In [28]:
bbp_df[cols] = bbp_df[cols].fillna(0)

/Users/chiranhewawitharana/mambaforge/envs/mactf/lib/python3.8/site-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [29]:
scaler = MinMaxScaler()
for col in cols:
    bbp_df[col] = scaler.fit_transform(bbp_df[col].values.reshape(-1,1))

/var/folders/00/gmvdw06n48lgtsz126ng5y2m0000gn/T/ipykernel_24001/277134799.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bbp_df[col] = scaler.fit_transform(bbp_df[col].values.reshape(-1,1))
/var/folders/00/gmvdw06n48lgtsz126ng5y2m0000gn/T/ipykernel_24001/277134799.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bbp_df[col] = scaler.fit_transform(bbp_df[col].values.reshape(-1,1))
/var/folders/00/gmvdw06n48lgtsz126ng5y2m0000gn/T/ipykernel_24001/277134799.py:3: SettingWithCopyWarning: 
A value is try

In [30]:
# Droping due to all null
bbp_df.dropna(how="all",axis=0,inplace=True)
bbp_df.dropna(how="all",axis=1,inplace=True)

/Users/chiranhewawitharana/mambaforge/envs/mactf/lib/python3.8/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [31]:
bbp_df.rename(columns={"Product_DB_ID":"package"},inplace=True)

/Users/chiranhewawitharana/mambaforge/envs/mactf/lib/python3.8/site-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [32]:
bbp_df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59 entries, 0 to 58
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   package              59 non-null     object 
 1   Base_Type            59 non-null     object 
 2   Pricing_Type         59 non-null     object 
 3   Package_Type         59 non-null     object 
 4   Title                59 non-null     object 
 5   BB_Data_Standard     59 non-null     float64
 6   BB_Data_Free         59 non-null     float64
 7   BB_Data_Anytime      59 non-null     float64
 8   BB_Data_Unlimited    59 non-null     float64
 9   BB_Connection_Type   59 non-null     object 
 10  BB_Connection_Speed  59 non-null     object 
 11  Price                59 non-null     object 
 12  Subscription_Type    59 non-null     object 
 13  Recidence_Type       59 non-null     object 
 14  Tax_Status           59 non-null     object 
 15  Conditions           59 non-null     objec

In [33]:
bbp_df.to_csv("data/azure/BB_package_content.csv")